# EDA notebook for the Iowa liquor sales dataset

In [63]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

## Data import and basic checks

In [81]:
sales_df = pd.read_csv('../raw_data/Iowa_liquor_sales_2021.csv', nrows= 100000)
sales_df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-37974400176,2021-07-01,2572,Hy-Vee Food Store / Cedar Falls,6301 University,Cedar Falls,50613.0,POINT (-92.435236 42.512789),7.0,BLACK HAWK,...,86507,Paramount Triple Sec,12,1000,3.84,5.76,4,23.04,4.0,1.05
1,INV-33767900019,2021-01-25,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854),85.0,STORY,...,904563,SOOH Black Haus Blackberry Schnapps,12,1000,15.00,22.50,4,90.00,4.0,1.05
2,INV-33739700007,2021-01-25,6076,Shop N Save Newton,404 S 11th Ave W,Newton,50208.0,POINT (-93.058352 41.690355),50.0,JASPER,...,101187,Hennessy VS NBA 2020 Giftbox,12,750,20.99,31.49,60,1889.40,45.0,11.88
3,INV-37792000067,2021-06-24,5102,Wilkie Liquors,724 1st St SE,Mount Vernon,52314.0,POINT (-91.410401 41.918328),57.0,LINN,...,86251,Juarez Triple Sec,12,1000,2.42,3.63,60,217.80,60.0,15.85
4,INV-34644200110,2021-03-01,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411.0,POINT (-91.701581 42.030129),57.0,LINN,...,86843,Ole Smoky Salty Caramel Whiskey,6,750,10.00,15.00,18,270.00,13.5,3.56


In [3]:
sales_df.dtypes

invoice_and_item_number     object
date                        object
store_number                 int64
store_name                  object
address                     object
city                        object
zip_code                   float64
store_location              object
county_number              float64
county                      object
category                   float64
category_name               object
vendor_number              float64
vendor_name                 object
item_number                  int64
item_description            object
pack                         int64
bottle_volume_ml             int64
state_bottle_cost          float64
state_bottle_retail        float64
bottles_sold                 int64
sale_dollars               float64
volume_sold_liters         float64
volume_sold_gallons        float64
dtype: object

In [67]:
sales_df.isna().sum()

invoice_and_item_number        0
date                           0
store_number                   0
store_name                     0
address                        1
city                           1
zip_code                       1
store_location             11603
county_number                  1
county                         1
category                       0
category_name                  0
vendor_number                  1
vendor_name                    1
item_number                    0
item_description               0
pack                           0
bottle_volume_ml               0
state_bottle_cost              0
state_bottle_retail            0
bottles_sold                   0
sale_dollars                   0
volume_sold_liters             0
volume_sold_gallons            0
dtype: int64

In [82]:
# drop NaNs
sales_df.drop(sales_df[sales_df['vendor_number'].isna() == True].index[0], axis = 0, inplace = True)
sales_df.drop(sales_df[sales_df['zip_code'].isna() == True].index[0], axis = 0, inplace= True)

In [83]:
# features that need their dtype to be changed: date, zip_code, county_number, category, vendor_number

sales_df['date'] = pd.to_datetime(sales_df['date'])
sales_df['zip_code'] = [int(x) for x in sales_df['zip_code']]
sales_df['county_number'] = [int(x) for x in sales_df['county_number']]
sales_df['category'] = [int(x) for x in sales_df['category']]
sales_df['vendor_number'] = [int(x) for x in sales_df['vendor_number']]

In [85]:
# Convert store_location into store_latitude and store_longitude
sales_df['store_location'] = [str(x).strip('POINT (').strip(')').split() for x in sales_df['store_location']]

In [90]:
sales_df['store_longitude'] = [round(float(x[0]),6) if len(x) > 1 else 0 for x in sales_df['store_location']]
sales_df['store_latitude'] = [round(float(x[1]),6) if len(x) > 1 else 0 for x in sales_df['store_location']]

In [91]:
sales_df.dtypes

invoice_and_item_number            object
date                       datetime64[ns]
store_number                        int64
store_name                         object
address                            object
city                               object
zip_code                            int64
store_location                     object
county_number                       int64
county                             object
category                            int64
category_name                      object
vendor_number                       int64
vendor_name                        object
item_number                         int64
item_description                   object
pack                                int64
bottle_volume_ml                    int64
state_bottle_cost                 float64
state_bottle_retail               float64
bottles_sold                        int64
sale_dollars                      float64
volume_sold_liters                float64
volume_sold_gallons               

In [8]:
# Check for duplicates
sales_df.shape

(99998, 24)

In [9]:
sales_df.invoice_and_item_number.nunique()

99998

In [10]:
sales_df.duplicated().sum()

0

In [11]:
# Check if basic stats make sense

In [12]:
sales_df.describe()

,store_number,zip_code,county_number,category,vendor_number,item_number,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
count,99998.000000,99998.000000,99998.000000,9.999800e+04,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000
mean,4137.196894,51243.400378,57.333837,1.053055e+06,310.154843,56275.150513,11.923828,883.299446,10.443469,15.666544,13.714424,162.427949,11.178018,2.947439
std,1250.473997,998.750411,27.137010,7.757280e+04,134.882736,95799.139294,8.449831,564.949000,9.122365,13.683999,41.365568,558.056128,53.874194,14.232244
min,2106.000000,50002.000000,1.000000,1.011100e+06,33.000000,258.000000,1.000000,20.000000,0.890000,1.340000,1.000000,1.350000,0.020000,0.000000
25%,2648.000000,50314.000000,31.000000,1.031100e+06,259.000000,35926.000000,6.000000,375.000000,5.420000,8.130000,3.000000,45.720000,2.250000,0.590000
50%,4198.500000,51034.000000,63.000000,1.032100e+06,301.000000,42566.000000,12.000000,750.000000,8.490000,12.740000,6.000000,81.000000,9.000000,2.370000
75%,5226.000000,52302.000000,77.000000,1.071100e+06,421.000000,64858.000000,12.000000,1000.000000,12.670000,19.010000,12.000000,162.000000,10.500000,2.770000
max,9048.000000,57222.000000,99.000000,1.901200e+06,978.000000,996237.000000,60.000000,3500.000000,330.500000,495.750000,3510.000000,43910.100000,6142.500000,1622.670000


In [13]:
# Check for redundant information
sales_df['county'] = [x.lower() for x in sales_df['county']]
sales_df['city'] = [x.lower() for x in sales_df['city']]

## Exploratory visualization

In [14]:
sales_df.dtypes

invoice_and_item_number            object
date                       datetime64[ns]
store_number                        int64
store_name                         object
address                            object
city                               object
zip_code                            int64
store_location                     object
county_number                       int64
county                             object
category                            int64
category_name                      object
vendor_number                       int64
vendor_name                        object
item_number                         int64
item_description                   object
pack                                int64
bottle_volume_ml                    int64
state_bottle_cost                 float64
state_bottle_retail               float64
bottles_sold                        int64
sale_dollars                      float64
volume_sold_liters                float64
volume_sold_gallons               

In [15]:
top_categories = sales_df.groupby('category_name', as_index=False).count()[['category_name', 'invoice_and_item_number']].sort_values('invoice_and_item_number', ascending=False)

In [99]:
px.bar(top_categories, x='category_name', y='invoice_and_item_number', title='Most ordered alcohol categories')

In [17]:
top_buyers_county = sales_df.groupby('county', as_index=False).count()[['county', 'invoice_and_item_number']].sort_values('invoice_and_item_number', ascending=False)

In [98]:
px.bar(top_buyers_county, x='county', y='invoice_and_item_number', title="Total invoices per county")

In [94]:
top_buyers_dollars_county = sales_df.groupby('county', as_index=False).sum()[['county', 'sale_dollars']].sort_values('sale_dollars', ascending=False)

In [97]:
px.bar(top_buyers_dollars_county, x='county', y='sale_dollars', title="Total sales per county in dollars")

In [113]:
date_sorted_sales = sales_df.sort_values('date')
date_sorted_sales.set_index('date', inplace=True)

In [114]:
date_sorted_sales

,invoice_and_item_number,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,...,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,store_longitude,store_latitude
date,,,,,,,,,,,,,,,,,,,,,
2020-12-02,INV-32382600017,2596,Hy-Vee Ottumwa#2,2453 North Court,Ottumwa,52501,[nan],90,WAPELLO,1011100,...,12,750,7.50,11.25,2,22.50,1.50,0.39,0.000000,0.000000
2020-12-02,INV-32370200032,5845,Casey's General Store #3827- Ankeny,3605 NW Irvinedale Drive,Ankeny,50023,"[-93.638982, 41.759957]",77,Polk,1081600,...,1,50,25.80,38.70,2,90.00,0.10,0.02,-93.638982,41.759957
2020-12-02,INV-32371000005,5080,C's Liquor Store,719 2nd Ave W,Spencer,51301,"[-95.147741, 43.14521]",21,CLAY,1051100,...,6,1750,11.33,17.00,6,102.00,10.50,2.77,-95.147741,43.145210
2020-12-02,INV-32387700055,2521,Hy-Vee Food and Drug / Grand / WDM,1990 Grand Avenue,West Des Moines,50265,"[-93.73162, 41.571127]",77,POLK,1011100,...,12,50,14.00,21.00,2,42.00,0.10,0.02,-93.731620,41.571127
2020-12-02,INV-32379100011,3041,Shugar's / Colfax,28 E Howard,Colfax,50054,"[-93.244443, 41.677932]",50,JASPER,1011100,...,12,375,3.50,5.25,3,15.75,1.12,0.29,-93.244443,41.677932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-30,INV-42455600047,5244,The Ox & Wren Spirits and Gifts,708 2nd AVE SE,Cresco,52136,"[-92.106529, 43.371131]",45,HOWARD,1031100,...,6,1750,9.44,14.16,2,28.32,3.50,0.92,-92.106529,43.371131
2021-11-30,INV-42477000001,5263,Bernie's Booze LLC,"950, West View Dr",Rock Valley,51247,[nan],84,SIOUX,1071100,...,1,1800,54.20,81.30,1,81.30,1.80,0.47,0.000000,0.000000
2021-11-30,INV-42454600008,4321,Fareway Stores #501 / Charles City,210 11th St,Charles City,50616,"[-92.674449, 43.052712]",34,FLOYD,1062500,...,12,750,7.00,10.50,4,42.00,3.00,0.79,-92.674449,43.052712


In [129]:
n_orders_per_month = date_sorted_sales.groupby(pd.Grouper(freq="M")).count()[['invoice_and_item_number']]
n_orders_per_day = date_sorted_sales.groupby(pd.Grouper(freq="D")).count()[['invoice_and_item_number']]
total_sales_per_day = date_sorted_sales.groupby(pd.Grouper(freq="D")).sum()[['sale_dollars']]

In [130]:
n_orders_per_month = n_orders_per_month.reset_index()
n_orders_per_day = n_orders_per_day.reset_index()
n_orders_per_month

,date,invoice_and_item_number
0,2020-12-31,9446
1,2021-01-31,7497
2,2021-02-28,7278
3,2021-03-31,8654
4,2021-04-30,8551
5,2021-05-31,8106
6,2021-06-30,9309
7,2021-07-31,8341
8,2021-08-31,8342
9,2021-09-30,8219


In [131]:
total_sales_per_month = total_sales_per_month.reset_index()
total_sales_per_day = total_sales_per_day.reset_index()
total_sales_per_month

,index,date,sale_dollars
0,0,2020-12-31,1590560.99
1,1,2021-01-31,1150755.69
2,2,2021-02-28,1204664.95
3,3,2021-03-31,1420473.08
4,4,2021-04-30,1242976.09
5,5,2021-05-31,1266609.72
6,6,2021-06-30,1460573.95
7,7,2021-07-31,1388882.46
8,8,2021-08-31,1355477.13
9,9,2021-09-30,1330944.06


In [127]:
px.line(n_orders_per_month, x='date', y='invoice_and_item_number', title='Monthly orders (all counties)')

In [132]:
# Stores do not order on weekends, liquor vendors not working
px.line(n_orders_per_day, x='date', y='invoice_and_item_number', title='Daily orders (all counties)')

In [128]:
px.line(total_sales_per_month, x='date', y='sale_dollars', title='Monthly sales (all counties)')

In [133]:
px.line(total_sales_per_day, x='date', y='sale_dollars', title='Daily sales (all counties)')